## Import packages

In [20]:
import numpy as np
from games.tictactoe import *
from games.common import TwoPlayersGameState

## Explore API of the game

In [21]:
state = np.zeros((3,3))
initial_board_state = TicTacToeGameState(state = state, next_to_move = 1)
initial_board_state

In [22]:
# return possible move
possible_actions = initial_board_state.get_legal_actions()
possible_actions

[x:0 y:0 v:1,
 x:0 y:1 v:1,
 x:0 y:2 v:1,
 x:1 y:0 v:1,
 x:1 y:1 v:1,
 x:1 y:2 v:1,
 x:2 y:0 v:1,
 x:2 y:1 v:1,
 x:2 y:2 v:1]

In [23]:
# take a move
current_state = initial_board_state.move(possible_actions[0])
current_state

In [24]:
current_state.next_to_move

-1

In [25]:
current_state.get_legal_actions()

[x:0 y:1 v:-1,
 x:0 y:2 v:-1,
 x:1 y:0 v:-1,
 x:1 y:1 v:-1,
 x:1 y:2 v:-1,
 x:2 y:0 v:-1,
 x:2 y:1 v:-1,
 x:2 y:2 v:-1]

In [26]:
print(current_state.is_game_over())
result = current_state.game_result

False


In [27]:
while(True):
    current_state = current_state.move(current_state.get_legal_actions()[-1])
    if current_state.is_game_over():
        print(current_state.game_result)
        break

-1.0


# Start with defining a node

In [46]:
class Node:
    
    def __init__(self, state: TwoPlayersGameState, parent = None):
        self.state = state
        self.parent = parent
        
        self.children = []
        self.actions = self.state.get_legal_actions()
        
        self.num_visit = 0 # number of visits
        self.results = {1: 0, -1: 0, 0: 0} # for each side
        
    # 1. Define three properties: Actions, Q and N
    @property
    def Q(self):
        # decide which side of player;
        this_side = self.parent.state.next_to_move
        other_side = this_side * -1
        # calculate Q
        wins = self.results[this_side]
        loses = self.results[other_side]
        # return sum
        return wins - loses

    # 2. Define checks
    def is_terminal_node(self):
        return self.state.is_game_over()

    def is_fully_expanded(self):
        return len(self.actions) == 0

    # 3. Define Action 1 - Roll Out
    def rollout(self):    
        # run simulation until game over, if not terminal
        state_now = self.state
        while not state_now.is_game_over():
            possible_actions = state_now.get_legal_actions()
            random_index = np.random.randint(len(possible_actions))
            random_action = possible_actions[random_index]
            state_now = state_now.move(random_action)

        return int(state_now.game_result)

    # 4. Define Action - 2: Backpropagation
    def backpropagate(self, who_wins):
        self.num_visit += 1
        self.results[who_wins] += 1
        if self.parent:
            self.parent.backpropagate(who_wins)

    # 5. Define how to expand a node by taking unused actions
    def expand(self):
        # 5.1 Add new Child
        new_action = self.actions.pop() # no need to have property setter
        child_state = self.state.move(new_action)
        child_node = Node(child_state, parent = self)
        self.children.append(child_node)

        # 5.2 Roll out this Child node and backpropagate result
        who_wins = child_node.rollout()
        child_node.backpropagate(who_wins)

    # 6. Define how to select best child for an expanded node
    def get_best_child(self, c_param):
        best_weight = -100
        for child in self.children:
            weight = child.Q / child.num_visit + \
                     c_param * np.sqrt((2 * np.log(self.num_visit) / (child.num_visit)))
            if weight > best_weight:
                best_child = child
        return best_child

# Then define how to find the next node

![image](https://int8.io/wp-content/uploads/2018/03/uct.png)

In [57]:
# Start Game
root = Node(initial_board_state)
current_node = root
while not current_node.is_terminal_node():
    if not current_node.is_fully_expanded():
        print('Expanding a Node')
        current_node.expand()
    else:
        print('Finishing expanding, finding best child')
        current_node = current_node.get_best_child(c_param = 0)
        
best_next_state = root.get_best_child(c_param = 0.5)

Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child
Expanding a Node
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child
Expanding a Node
Expanding a Node
Expanding a Node
Finishing expanding, finding best child


# Reference

https://int8.io/monte-carlo-tree-search-beginners-guide/#Monte_Carlo_Tree_Search_8211_basic_concepts